In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd

%load_ext rpy2.ipython

In [2]:
%%R 

suppressPackageStartupMessages({
    library(cowplot)
    library(tidyverse)
    library(theft)
})


# Set cowplot theme
theme_set(theme_cowplot())


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [48]:
parc_info = pd.read_csv("/Users/abry4213/data/Rest_vs_Film_fMRI/parc_info.csv").assign(row_index = lambda x: x.index)

In [68]:
ts_file_path = "/Users/abry4213/data/Rest_vs_Film_fMRI/time_series_files"

TS_data_7_networks_list = []

for ts_file in os.listdir(ts_file_path):
    sample_ID, scan_type, scan_session, _ = (ts_file.split("."))
    this_data = (pd.read_csv(f"{ts_file_path}/{ts_file}", header=None, na_values="nan")
                 .assign(timepoint = lambda x: x.index + 1)
                 .melt(id_vars=["timepoint"], var_name="row_index", value_name="values")
                 .merge(parc_info, on="row_index", how="left"))
    this_data_7_networks = (this_data.groupby(["Yeo_7_Networks_Unilateral", "timepoint"], as_index=False).agg({"values": "mean"}))
    this_data_7_networks["sample_ID"] = sample_ID
    this_data_7_networks["scan_type"] = scan_type
    this_data_7_networks["scan_session"] = scan_session
    this_data_7_networks["Unique_ID"] = f"{sample_ID}_{scan_type}_{scan_session}"

    TS_data_7_networks_list.append(this_data_7_networks)

TS_data_7_networks =  pd.concat(TS_data_7_networks_list).reset_index(drop=True)
TS_data_7_networks_mean_and_SD = TS_data_7_networks.groupby(["sample_ID", "scan_type", "scan_session"])['values'].agg(["mean", "std"]).reset_index()


How many participants have valid scans for all four sessions for both movie and rest?

In [74]:
%%R -i TS_data_7_networks_mean_and_SD
TS_data_7_networks_mean_and_SD %>%
    filter(!is.na(mean)) %>%
    group_by(sample_ID, scan_type) %>%
    count() %>%
    group_by(sample_ID) %>%
    filter(all(n==4)) %>%
    distinct(sample_ID)

# A tibble: 23 × 1
# Groups:   sample_ID [23]
   sample_ID
   <chr>    
 1 sub01    
 2 sub02    
 3 sub04    
 4 sub06    
 5 sub07    
 6 sub08    
 7 sub09    
 8 sub13    
 9 sub14    
10 sub15    
# ℹ 13 more rows
# ℹ Use `print(n = ...)` to see more rows


How many participants have valid scans for all four session for each scan type separately?

In [76]:
%%R -i TS_data_7_networks_mean_and_SD
TS_data_7_networks_mean_and_SD %>%
    filter(!is.na(mean)) %>%
    group_by(sample_ID, scan_type) %>%
    count() %>%
    filter(n==4) %>%
    group_by(scan_type) %>%
    count()

# A tibble: 2 × 2
# Groups:   scan_type [2]
  scan_type     n
  <chr>     <int>
1 movie        25
2 rest         27


In [29]:
%%R -i TS_data_7_networks -o catch24_7_networks

catch24_7_networks = calculate_features(data = TS_data_7_networks, 
                    id_var = "Unique_ID",
                   feature_set = "catch22",
                   features = list("mean" = mean, "sd" = sd))

Removed ID: sub03_movie_3 due to non-real values.
Removed ID: sub03_movie_4 due to non-real values.
Removed ID: sub05_rest_3 due to non-real values.
Removed ID: sub10_movie_3 due to non-real values.
Removed ID: sub10_movie_4 due to non-real values.
Removed ID: sub11_movie_3 due to non-real values.
Removed ID: sub11_movie_4 due to non-real values.
Removed ID: sub12_rest_1 due to non-real values.
Removed ID: sub12_rest_3 due to non-real values.
Removed ID: sub29_movie_4 due to non-real values.
Total IDs removed due to non-real values: sub03_movie_3 (4%)Total IDs removed due to non-real values: sub03_movie_4 (4%)Total IDs removed due to non-real values: sub05_rest_3 (4%)Total IDs removed due to non-real values: sub10_movie_3 (4%)Total IDs removed due to non-real values: sub10_movie_4 (4%)Total IDs removed due to non-real values: sub11_movie_3 (4%)Total IDs removed due to non-real values: sub11_movie_4 (4%)Total IDs removed due to non-real values: sub12_rest_1 (4%)Total IDs removed due to 

In [31]:
catch24_7_networks.reset_index(drop=True).to_feather("/Users/abry4213/data/Rest_vs_Film_fMRI/catch24_7_networks.feather")